#### 라이브러리

In [ ]:
## selenium 설치 
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 968 kB 5.6 MB/s 
     |████████████████████████████████| 359 kB 30.2 MB/s 
     |████████████████████████████████| 138 kB 34.1 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 4.0 MB 26.5 MB/s 
     |████████████████████████████████| 58 kB 4.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu

In [ ]:
import requests
from bs4 import BeautifulSoup
import time 
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#셀레니움으로 크롬 열기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')   ## 브라우저가 눈에 안보여도 조작 가능하게
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

### 홈페이지 html parsing

In [ ]:
page = 1 
url = f'https://www.oliveyoung.co.kr/store/main/getSaleList.do?dispCatNo=900000100090001&fltDispCatNo=10000010001&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb'  # 올리브영 - 세일 카테고리
raw = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(raw.text, 'html.parser')
#print(html)

In [ ]:
# webdriver 실행
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(url)
#driver.find_element(By.CLASS_NAME, "pagingNumber-box.box").find_element(By.CLASS_NAME, 'totalPagingNum').text.split('/')[1].strip()
sdf

NameError: ignored

In [ ]:
total_num = html.select('p.cate_info_tx span')[0].text.strip() #전체 세일 등록 상품 개수

In [ ]:
last_page = int(total_num) // 48 + 1
print(last_page)
#page = range(1,last_page+1)

17


In [ ]:
# 상품 카테고리 
category = html.select('div#Container div.common-menu li')[1:19]
key = category[0].text
value = category[0].find('button')['data-ref-dispcatno']
print(key, value)

print(len(category))
category[1].text

스킨케어 10000010001
18


'마스크팩'

In [ ]:
## 전체 카테고리 넘버링
# for i in range(len(category)):
#     value = category[i].find('button')['data-ref-dispcatno']  # 카테고리 넘버
#     key = category[i].text   #카테고리명
#     print(key, value)

In [ ]:
products = html.select('ul.cate_prd_list.gtm_sale_list div.prd_info')
#print(len(products))

# 예시- 첫번째 상품
products[0].select('div.prd_name span')[0].text  # 회사명
products[0].select('div.prd_name p')[0].text  # 상품명
products[0].select('p.prd_price')[0].text.split()  # 정가, 세일가
#products[0].select('a.prd_thumb.goodsList')[0]['data-ref-itemno']  #상품 카테고리 
products[0].select('a')[0]['href']  # 상품 url 
# 전체 상품
# results = []
# for i in range(len(products)):
#     result = []
#     company = products[i].select('div.prd_name span')[0].text
#     product_name = products[0].select('div.prd_name p')[0].text
#     fixed_price , sale_price = products[0].select('p.prd_price')[0].text.split()

#     contents = {'company': company, 'product_name': product_name, 
#                 'price' : fixed_price, 'sale_price' : sale_price}
#     result.append(contents)

# results.extend(result)

'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000163539&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'

In [ ]:
url = products[0].select('a')[0]['href'] 

goods = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(goods.text, 'html.parser')

cat = html.select('ul.loc_history')[0].select('a.cate_y')
cat_1 = cat[0].text
cat_2 = cat[1].text
cat_3 = cat[2].text
print(cat_1, cat_2, cat_3)  #카테고리 대/중/소 분류

### 크롤링 함수
- 올리브영 세일 > 상품명, 카테고리, 정가/세일가, 세부페이지 url 

In [ ]:
def get_sale_products(url):        
    raw = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
    html = BeautifulSoup(raw.text, 'html.parser')

    # 전체 세일 상품 갯수 -> 마지막 페이지 수 구하기
    total_num = html.select('p.cate_info_tx span')[0].text.strip().replace(',','')
    page_num = int(total_num) // 48 + 1
    print(f"last_page_number: {page_num}")


    # 전체 페이지에 대한 세일 상품 추출
    results = []
    for i in range(1, page_num):
        result = []

        for i in range(len(products)):           
            # 회사명
            company = products[i].select('div.prd_name span')[0].text
            # 상품명
            product_name = products[i].select('div.prd_name p')[0].text
            # 가격(정가, 세일가)
            fixed_price , sale_price = products[i].select('p.prd_price')[0].text.split()

            # 상품 url (세부 정보 추출)
            url = products[i].select('a')[0]['href']

            # 상품 세부 카테고리 --------- 추가하면 런타임이 너무 길어짐
            # goods = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0'})
            # html = BeautifulSoup(goods.text, 'html.parser')

            # cat = html.select('ul.loc_history')[0].select('a.cate_y')
            # cat_1 = cat[0].text
            # cat_2 = cat[1].text
            # cat_3 = cat[2].text

            contents = {'company': company, 'product_name': product_name, 
                        'price' : fixed_price, 'sale_price' : sale_price, 'url' : url}
            
            result.append(contents)

        results.extend(result)
        
        # 어떤 데이터를 보낼까? 
        #text = results[0]
        ## 카카오톡 알림
        #send_to_kakao(text)
    
    return results


### 카카오톡 알림 설정( 다시 ... 오류..)
https://www.youtube.com/watch?v=gg2sjQ0877U

In [ ]:
# KAKAO_TOKEN = '2d9d00cc5dbd4174e663ee2de28a275b'

# def send_to_kakao(text):
#     header = {'Authorization':'Bearer'+ KAKAO_TOKEN}
#     url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'
#     post = {"object_type": text,
#         "text": "텍스트 영역입니다. 최대 200자 표시 가능합니다.",
#         "link": {
#             "web_url": "https://developers.kakao.com",
#             "mobile_web_url": "https://developers.kakao.com"
#         },
#         }
#     data = {'template_object': json.dumps(post)}
#     requests.post(url, headers = header, data = data)

### 크롤링 결과 데이터 프레임으로 만들기 

In [ ]:

from typing import Dict
import pandas as pd

def to_csv(data: Dict):
    df = pd.DataFrame(data).to_csv('/content/sample_data/olive_young_sale.csv', index = False)
    return df

### 함수 실행 결과

In [ ]:
#page = 1 
#url = f'https://www.oliveyoung.co.kr/store/main/getSaleList.do?dispCatNo=900000100090001&fltDispCatNo=10000010001&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb'  # 올리브영 - 세일 카테고리(스킨)
url = f'https://www.oliveyoung.co.kr/store/main/getSaleList.do?dispCatNo=900000100090001&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb'  # 올리브영 - 세일 카테고리(전체)
result = get_sale_products(url)
print(result)
print(len(result))

last_page_number: 161
[{'company': '아이소이', 'product_name': '[한정기획] 아이소이 잡티세럼 업 40ml 기획 (+15ml)', 'price': '54,000원', 'sale_price': '37,800원', 'url': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000163539&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'}, {'company': '아누아', 'product_name': '아누아 어성초 77 수딩 토너 350ml 기획 (+40ml +앰플10ml)', 'price': '30,500원', 'sale_price': '22,800원', 'url': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000162254&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'}, {'company': '아비브', 'product_name': '[단독기획]아비브 어성초 카밍 토너 스킨부스터 더블 기획 (200ml+200ml)', 'price': '39,000원', 'sale_price': '27,300원', 'url': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000155253&dispCatNo&trackingCd=Sale_Pop&curation&egcode&rccode&egrankcode'}, {'company': '닥터지', 'product_name': '[2세대 천만크림] 닥터지 레드 블레미쉬 클리어 수딩 크림 70ml+30ml 세트', 'price': '36,000원', 'sale_price': '25,200원', 'url': 

In [ ]:
df = to_csv(result)
df = pd.read_csv('/content/sample_data/olive_young_sale.csv')
df.shape

(7680, 5)

## 리뷰 추출

In [ ]:
df2 = df[['company', 'product_name', 'url']]
#df2.groupby('company').count()
df.query('company == "가히"' )

,company,product_name,price,sale_price,url
18,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
66,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
114,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
162,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
210,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
...,...,...,...,...,...
7458,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
7506,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
7554,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
7602,가히,가히 멀티밤 & 멀티밤 듀오,"75,600원","58,800원",https://www.oliveyoung.co.kr/store/goods/getGo...
